# QuakeMigrate - Example - Icequake detection

## Overview:

This notebook shows how to run QuakeMigrate for icequake detection, using a 2 minute window of continuous seismic data from Hudson et al (2019). Please refer to this paper for details and justification of the settings used.

Here, we detail how to:
1. Create a travel-times lookup table for the example seismometer network
2. Run the detect stage to coalesce energy through time
3. Run the trigger stage to determine events above a threshold value
4. Run the locate stage to refine the earthquake location

We also provide an outline of some of the key outputs

In [33]:
# Import necessary modules:
import QMigrate.core.model as qmod
import QMigrate.signal.scan as qscan
import QMigrate.io.data as qdata
import QMigrate.io.quakeio as qio
import QMigrate.signal.trigger as qtrigger

In [34]:
# Set i/o paths:
station_file = "./inputs/stations.txt"
data_in   = "./inputs/mSEED"
lut_out   = "./outputs/lut/icequake.LUT"
out_path  = "./outputs/runs"
run_name  = "icequake_example"

## 1. Create a travel-times lookup table (LUT)

In [35]:
# Read in station information
stations = qio.stations(station_file)

# Set the parameters for the travel-times lookup table (LUT)
# Cell count (x,y,z); cell size (x,y,z in metres)
lut = qmod.LUT(stations, cell_count=[20, 20, 140], cell_size=[100, 100, 20])
lut.lonlat_centre(-17.224, 64.328)

# Set the LUT projection (here we use the Lambert Conformal Conic projection)
lut.lcc_standard_parallels = (64.32, 64.335)
lut.projections(grid_proj_type="LCC")
lut.elevation=1400 # Defining the elevation of the top of the grid in m 

# Compute for a homogeneous velocity model
v_p_homo_model = 3630
v_s_homo_model = 1833
lut.compute_homogeneous_vmodel(v_p_homo_model, v_s_homo_model)

# Save the LUT
lut.save(lut_out)

## 2. Coalesce the seismic energy through time

In [37]:
# Create a new instance of the MSEED class and set path structure
data = qdata.Archive(station_file=station_file, archive_path=data_in)
data.path_structure(archive_format="YEAR/JD/*_STATION_*")

# Create a new instance of the SeisScan class
scan = qscan.QuakeScan(data, lut_out, output_path=out_path, run_name=run_name)

	QuakeMigrate - Coalescence Scanning - Path: outputs/runs - Name: icequake_example



In [38]:
# Set detect parameters
scan.sampling_rate = 500           # Sampling rate of data, in Hz
scan.p_bp_filter   = [10, 125, 4]  # The band-pass filter parameters for the P-phase (10 to 125 Hz, with 4th order corners)
scan.s_bp_filter   = [10, 125, 4]  # The band-pass filter parameters for the P-phase (10 to 125 Hz, with 4th order corners)
scan.p_onset_win   = [0.01, 0.25]  # Length of the STA and LTA time windows for the P-phase
scan.s_onset_win   = [0.05, 0.5]   # Length of the STA and LTA time windows for the S-phase
scan.time_step     = 0.75          # The length of the time-step
scan.decimate      = [1, 1, 1]     # Decimation factors in x,y,z (no decimation here)
scan.n_cores       = 12            # Number of cores/processors to use

# Defining the start and end times 
starttime = "2014-06-29T18:41:55.0"
endtime   = "2014-06-29T18:42:20.0"

In [39]:
# Run the detect stage to find the coalescence of energy through time:
scan.detect(starttime, endtime)

	DETECT - Continuous Seismic Processing

	Parameters specified:
		Start time                = 2014-06-29T18:41:55.000000Z
		End   time                = 2014-06-29T18:42:20.000000Z
		Time step (s)             = 0.75
		Number of CPUs            = 12

		Sampling rate             = 500
		Grid decimation [X, Y, Z] = [1, 1, 1]
		Bandpass filter P         = [10, 125, 4]
		Bandpass filter S         = [10, 125, 4]
		Onset P [STA, LTA]        = [0.01, 0.25]
		Onset S [STA, LTA]        = [0.05, 0.5]

~~~~~~~~~~~~~~~~~~~~~~~~ Processing : 2014-06-29T18:41:53.350000Z - 2014-06-29T18:42:01.750000Z ~~~~~~~~~~~~~~~~~~~~~~~~
    	Elapsed time: 0.575300 seconds.

~~~~~~~~~~~~~~~~~~~~~~~~ Processing : 2014-06-29T18:41:54.100000Z - 2014-06-29T18:42:02.500000Z ~~~~~~~~~~~~~~~~~~~~~~~~
    	Elapsed time: 0.602198 seconds.

~~~~~~~~~~~~~~~~~~~~~~~~ Processing : 2014-06-29T18:41:54.850000Z - 2014-06-29T18:42:03.250000Z ~~~~~~~~~~~~~~~~~~~~~~~~
    	Elapsed time: 0.605471 seconds.

~~~~~~~~~~~~~~~~~~~~~~~~ Pro

## 3. Run the trigger stage, to detect and output individual icequakes

nb: We can use the same SeisScan object here because we are not using a different decimation. If running trigger and locate on grids with different levels of decimation, a new SeisScan object must be initialised.

In [41]:
trig = qtrigger.Trigger(out_path, run_name, stations)

trig.normalise_coalescence = True
trig.marginal_window = 2.75
trig.minimum_repeat = 6.
trig.detection_threshold = 1.8

# Run trigger
trig.trigger(starttime, endtime, savefig=True)

   TRIGGER - Triggering events from coalescence

   Parameters specified:
         Start time                = 2014-06-29T18:41:55.000000Z
         End   time                = 2014-06-29T18:42:20.000000Z
         Pre/post pad              = 120 s

         Detection threshold       = 1.8
         Marginal window           = 2.75 s
         Minimum repeat            = 6.0 s

         Trigger from normalised coalescence - True

    Reading in scanmseed...

		Successfully read .scanmseed data from 2014-06-29T18:41:55.000000Z - 2014-06-29T18:42:20.498000Z

	Warning! No scanmseed data found for pre-pad!
	Warning! No scanmseed data found for post-pad!

    scanmseed read complete.

    Triggering...
	Triggered event 1 of 1

	Plotting triggered events on decimated grid...
		Successfully read .StationAvailability data from 2014-06-29T18:41:55.000000Z - 2014-06-29T18:42:19.750000Z



## 4. Run the locate stage, to relocate triggered events on a less decimated grid

In [42]:
# Set locate parameters:
scan.marginal_window = 2.75

# Turn on plotting features
scan.plot_coal_video      = False
scan.plot_coal_grid       = False
scan.plot_coal_picture    = True
scan.plot_coal_trace      = False

In [43]:
# Run the locate stage to determine the location of any triggered events
scan.locate(starttime, endtime)

	LOCATE - Determining earthquake location and uncertainty

	Parameters specified:
		Start time                = 2014-06-29T18:41:55.000000Z
		End   time                = 2014-06-29T18:42:20.000000Z
		Number of CPUs            = 12


	EVENT - 1 of 1 - 20140629184209576000

	Determining event location...

	Reading waveform data...
    	Elapsed time: 0.122227 seconds.

	Computing 4D coalescence grid...
    	Elapsed time: 7.081238 seconds.

	Making phase picks...
    	Elapsed time: 0.028397 seconds.

	Determining earthquake location and uncertainty...
		Gridded loc: 10   12   94
		Spline  loc: 10.3 11.7 94.0
    	Elapsed time: 2.997333 seconds.

	Plotting event summary figure...
		Logo not plotting
    	Elapsed time: 2.014111 seconds.




## 4. Some of the key outputs

In [48]:
# Show the .event file, containing event origin time and location:
icequake_event_fname = "./outputs/runs/icequake_example/events/20140629184210330000.event"
with open(icequake_event_fname) as f:
    lines = f.readlines()
for line in lines:
    print(line)

DT,COA,X,Y,Z,LocalGaussian_X,LocalGaussian_Y,LocalGaussian_Z,LocalGaussian_ErrX,LocalGaussian_ErrY,LocalGaussian_ErrZ,GlobalCovariance_X,GlobalCovariance_Y,GlobalCovariance_Z,GlobalCovariance_ErrX,GlobalCovariance_ErrY,GlobalCovariance_ErrZ

2014-06-29T18:42:10.330000Z,1.5194944269355477,-17.22234553106984,64.32997339610299,500.0,-17.220890041926193,64.33017645210302,513.2986894145288,163.7976722561411,183.17135670506303,176.30955167459342,-17.22181919307945,64.33025256921235,502.7027366951547,106.96901290777376,86.88609844488155,118.11844833684806



In [49]:
# Show the .stn file, containing station time picks:
icequake_stn_fname = "outputs/runs/icequake_example/picks/20140629184210330000.picks"
with open(icequake_stn_fname) as f:
    lines = f.readlines()
for line in lines:
    print(line)

Name,Phase,ModelledTime,PickTime,PickError,SNR

SKR01,P,2014-06-29T18:42:10.565278Z,2014-06-29T18:42:10.537238Z,0.01290113502982887,2.8512868169316716

SKR01,S,2014-06-29T18:42:10.795935Z,2014-06-29T18:42:10.763598Z,0.05396774047398362,2.0951973108967037

SKR02,P,2014-06-29T18:42:10.561196Z,2014-06-29T18:42:10.546318Z,0.008994819471905014,2.726206673673068

SKR02,S,2014-06-29T18:42:10.787852Z,-1,-1,-1

SKR03,P,2014-06-29T18:42:10.589234Z,-1,-1,-1

SKR03,S,2014-06-29T18:42:10.843376Z,-1,-1,-1

SKR04,P,2014-06-29T18:42:10.604371Z,-1,-1,-1

SKR04,S,2014-06-29T18:42:10.873353Z,-1,-1,-1

SKR05,P,2014-06-29T18:42:10.591184Z,2014-06-29T18:42:10.598644Z,0.012307307090651028,2.662305896754175

SKR05,S,2014-06-29T18:42:10.847238Z,2014-06-29T18:42:10.897377Z,0.05688926610461391,2.0798144094242828

SKR06,P,2014-06-29T18:42:10.579875Z,2014-06-29T18:42:10.573207Z,0.007577927477811804,2.513855187574924

SKR06,S,2014-06-29T18:42:10.824842Z,2014-06-29T18:42:10.831171Z,0.060531262386360786,2.06566057858

In [50]:
# Show the coalescence pdf file, containing event origin time and location:
icequake_coal_image_fname = "outputs/runs/icequake_example/summaries/icequake_example_20140629184210330000_EventSummary.pdf"
from IPython.display import IFrame # For plotting pdf
IFrame(icequake_coal_image_fname, width=800, height=400) # Plot pdf

References:

Hudson, T.S., Smith, J., Brisbourne, A.M., and White R.S. (2019). Automated detection of basal icequakes and discrimination from surface crevassing. Annals of Glaciology, 79